In [1]:
# import designMethods.en_13001_3_3 as en
# from output import create_output_file
from IPython.display import display, FileLink
import ipywidgets as widgets
import pathlib
import os
import itertools
from marsDemonstrator import Main_application

In [2]:
# !pip install voila
# !jupyter serverextension enable voila --sys-prefix

In [3]:
en_13001 = Main_application()

In [4]:
def init_gps(change):
    msg_box.value = "Initializing GPs: this may take up to 30 seconds"

    # initialize prediction class
    # en_13001.computed = en.Computed_data()

    # get current config (1 mast or 2 mast)
    en_13001.input.config = drop_config.value

    en_13001.init_gps()

    # en_13001["input"].config_loaded = True
    en_13001.input.config_loaded = True
    msg_box.value = "Initialized GPs"

In [5]:
def start_computation(change):

    # if there were never inputs loaded throw error message 
    if en_13001.input_file_loaded is None:
        if uploader._counter == 0:
            msg_box.value = "Please upload an excel file"
            return
    
    # if there is a new upload file load its content and do make new computation
    if uploader._counter > 0:
        # initialize input class
        # en_13001.input = en.EN_input()

        # file to temporarily save input data to
        en_13001.input_file_path = pathlib.Path("./input.xlsx")
        with open(en_13001.input_file_path, "w+b") as i:
            i.write(uploader.data[-1])

        # read temp file and delete after
        en_13001.read_input_file(en_13001.input_file_path)

        os.remove(en_13001.input_file_path)

        # reset uploader after reading file
        uploader.value.clear()
        uploader._counter = 0
        
        # if gps for configuration were not loaded yet load them
        if en_13001.input_file_loaded is None:
            init_gps(None)
        msg_box.value = "Start Computation"

        en_13001.run_computation_and_create_output(1)

    msg_box.value = "Finished computation"

    # create output file
    local_file = FileLink(en_13001.outname, result_html_prefix="Click here to download results: ")

    # display output file and write errors to output if there are any
    out_file.clear_output()
    out_errors.clear_output()
    with out_file:
        display(local_file)
        display(btn_delete_output)
    with out_errors:
        for error in list(itertools.chain(*en_13001.input.error_report)):
            print(error)

In [6]:
def delete_output_file(change):
    os.remove(gen_vars["output_file"])
    out_file.clear_output()

In [7]:
## Create widtges
# uploader for input
uploader = widgets.FileUpload(accept=".xlsx", multiple=False)

# dropdown for computation mode
drop_mode = widgets.Dropdown(description="Computation Mode", options=["proof", "min_diameter"], value="proof")

# dropdown for configuration
drop_config = widgets.Dropdown(description="SC Configuration", options=["m1", "m2"], value="m1")

# button for updating configuration
btn_update_config = widgets.Button(description="Update Configuration")
btn_update_config.on_click(init_gps)

# button to start computation
btn_start = widgets.Button(description="Start computation")
btn_start.on_click(start_computation)

# output widget to display output file
out_file = widgets.Output()

# output widget to write error reports
out_errors = widgets.Output()

# message box for information on computation process
msg_box = widgets.Text(value="", description="Message Box")

# button to delete output file from server
btn_delete_output = widgets.Button(description="Delete Resultfile")
btn_delete_output.on_click(delete_output_file)

In [8]:
## Build gui
widgets.VBox([widgets.Label("MARS Demonstrator"),
              drop_mode, drop_config, btn_update_config, uploader, msg_box, btn_start, out_file, out_errors])